In [1]:
from flwr.client.mod import adaptiveclipping_mod
from flwr.server.strategy import (
    DifferentialPrivacyClientSideAdaptiveClipping,
    FedAvg,
)

from utils4 import *

/home/kareem/mambaforge/envs/fl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-11 03:28:53,598	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
def load_data(partition_id):
    fds = FederatedDataset(dataset="mnist", partitioners={"train": 10})
    partition = fds.load_partition(partition_id)

    traintest = partition.train_test_split(test_size=0.2, seed=42)
    traintest = traintest.with_transform(normalize)
    trainset, testset = traintest["train"], traintest["test"]

    trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
    testloader = DataLoader(testset, batch_size=64)
    return trainloader, testloader

In [3]:
class FlowerClient(NumPyClient):
    def __init__(self, net, trainloader, testloader):
        self.net = net
        self.trainloader = trainloader
        self.testloader = testloader

    def fit(self, parameters, config):
        set_weights(self.net, parameters)
        train_model(self.net, self.trainloader)
        return get_weights(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        set_weights(self.net, parameters)
        loss, accuracy = evaluate_model(self.net, self.testloader)
        return loss, len(self.testloader), {"accuracy": accuracy}


def client_fn(context: Context) -> Client:
    net = SimpleModel()
    partition_id = int(context.node_config["partition-id"])
    trainloader, testloader = load_data(partition_id=partition_id)
    return FlowerClient(net, trainloader, testloader).to_client()

In [4]:
client = ClientApp(
    client_fn,
    mods=[adaptiveclipping_mod],  # modifiers
)

# DP : Differential Privacy

In [5]:
net = SimpleModel()
params = ndarrays_to_parameters(get_weights(net))

In [6]:
net = SimpleModel()
params = ndarrays_to_parameters(get_weights(net))


def server_fn(context: Context):
    fedavg_without_dp = FedAvg(
        fraction_fit=0.6,
        fraction_evaluate=1.0,
        initial_parameters=params,
    )
    fedavg_with_dp = DifferentialPrivacyClientSideAdaptiveClipping(
        fedavg_without_dp,  # <- wrap the FedAvg strategy
        noise_multiplier=0.3,
        num_sampled_clients=6,
    )

    # Adjust to 50 rounds to ensure DP guarantees hold
    # with respect to the desired privacy budget
    config = ServerConfig(num_rounds=5)

    return ServerAppComponents(
        strategy=fedavg_with_dp,
        config=config,
    )

In [7]:
server = ServerApp(server_fn=server_fn)

In [8]:
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=10,
    backend_config=backend_setup,
)

INFO : Starting Flower ServerApp, config: num_rounds=5, no round_timeout
INFO : 
INFO : [INIT]
INFO : Using initial global parameters provided by strategy
INFO : Starting evaluation of initial global parameters
INFO : Evaluation returned no results (`None`)
INFO : 
INFO : [ROUND 1]
INFO : configure_fit: strategy sampled 2 clients (out of 10)
(ClientAppActor pid=35217) INFO : adaptiveclipping_mod: parameters are clipped by value: 0.1000.
INFO : aggregate_fit: received 2 results and 0 failures
INFO : aggregate_fit: central DP noise with 0.0051 stdev added
INFO : configure_evaluate: strategy sampled 10 clients (out of 10)
INFO : aggregate_evaluate: received 10 results and 0 failures
INFO : 
INFO : [ROUND 2]
INFO : configure_fit: strategy sampled 6 clients (out of 10)
(ClientAppActor pid=35217) INFO : adaptiveclipping_mod: parameters are clipped by value: 0.0889. [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https: